# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('Desktop/Springboard/Mini_Projects/EDA_racial_discrimination/data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [6]:
data.head()
data.columns
data.shape

(4870, 65)

<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>

</div>

## Q1. What test is appropriate for this problem? Does CLT apply?

Answer:

We will do a 'two sample z-test'.
CLT is applicable as the number of sample (4870) is sufficiently large and each observation is independent.

## Q2. What are the null and alternate hypotheses?

Answer:
Null hypothesis: race doesn't impact the callbacks rates for resumes; $$H_0: p_B = p_W$$
Alternate hypothesis: race impact the callback rates for resumes; $$H_A: p_B \ne p_W$$

## Q3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.


In [17]:
w = data[data.race=='w']
b = data[data.race=='b']
call_w = data[data.race=='w'].call
call_b = data[data.race=='b'].call

In [27]:
# Your solution to Q3 here
# frequentist statistical approach
# probability of callback for white, and black
p_w = np.sum(call_w)/len(call_w)
p_b = np.sum(call_b/len(call_b))
print('probability for whites:', p_w, 'probability for black:', p_b)
p_diff = p_w - p_b
p_hat = (np.sum(call_w) + np.sum(call_b)) / (len(w) + len(b))

z = p_diff / np.sqrt(p_hat * (1 - p_hat) * ((1 / len(w)) + (1 / len(b))))

pval = stats.norm.cdf(-z) * 2
print("Z score:", z)
print("P-value:", pval)

# margin of error (err_m) and confidence interval (conf_int)
err_m = 1.96 * np.sqrt(p_hat * (1 - p_hat) * ((1 / len(w)) + (1 / len(b))))
conf_int = p_diff + np.array([-1, 1]) * err_m
print("Margin of Error:", err_m)
print("Confidence interval:", conf_int)


probability for whites: 0.09650924024640657 probability for black: 0.064476386
Z score: 4.108412194032628
P-value: 3.983886120160611e-05
Margin of Error: 0.015281912310894095
Confidence interval: [0.01675094 0.04731477]


In [23]:
##Generating a permutation sample
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2


In [24]:
## Generating permutation replicates
def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates


In [36]:
def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = data_1.mean() - data_2.mean()

    return diff

# Compute difference of mean impact force from experiment: empirical_diff_means
empirical_diff_means = diff_of_means(call_w, call_b)

# Draw 10,000 permutation replicates: perm_replicates

perm_replicates = draw_perm_reps(call_w, call_b,
                                 diff_of_means, size=100000)
print('observed difference in means:', empirical_diff_means)

# Compute p-value: p
p = (np.sum(perm_replicates >= empirical_diff_means)) / len(perm_replicates)
# Print the result
print('p-value =', p)

observed difference in means: 0.03203285485506058
p-value = 2e-05


The p value is much lower than 0.05. So, we reject the null hypothesis that race does not impact that callbacl rate for resumes.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

## Q4. Write a story describing the statistical significance in the context of the original problem.

We first made a null hypothesis that the race does not influnce the callback rate for the resumes. Then we drew random permutation from the concatenated callback data of black and white sounding resumes to test how frequently we observe larger difference in means than we see here. We obtained a very small p-value suggesting it would be very rare, only twice out of 100000 times, actually. So, the null hypothesis is rejected. The race influences the rate of call back.  

## Q5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

For this analysis the only variable we considered is race/ name. The resumes were otherwise identical. But we don't have the data on the job for which those resumes were send. The job requirements could affect the call back rate.